In [1]:
%autoreload 2

In [2]:
from ast import literal_eval
import json
import os
import sys
import typing

import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src import fitness_energy_utils as utils
from src.fitness_features import *
from src.ast_counter_sampler import *
from src.evolutionary_sampler import *
from src import latest_model_paths

2024-05-29 16:18:59 - ast_utils - DEBUG    - Using cache folder: /Users/guydavidson/tmp/game_generation_cache


# Load data and model

In [3]:
grammar = open('../dsl/dsl.ebnf').read()
grammar_parser = tatsu.compile(grammar)
game_asts = list(cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', grammar_parser, False, relative_path='..'))
real_game_texts = [ast_printer.ast_to_string(ast, '\n') for ast in game_asts]


fitness_df = utils.load_fitness_data('../data/fitness_features_1024_regrowths.csv.gz')
print(fitness_df.src_file.unique())
fitness_df.head()

2024-05-29 16:19:03 - ast_utils - INFO     - Loading from cache file: /Users/guydavidson/tmp/game_generation_cache/interactive-beta-cache.pkl.gz
2024-05-29 16:19:03 - ast_utils - INFO     - Finished loading cache file: /Users/guydavidson/tmp/game_generation_cache/interactive-beta-cache.pkl.gz


['interactive-beta.pddl' 'ast-real-regrowth-samples-1024.pddl.gz']


,Index,src_file,game_name,domain_name,original_game_name,real,variables_defined_all,variables_defined_prop,variables_used_all,variables_used_prop,...,ast_ngram_constraints_n_4_score,ast_ngram_constraints_n_5_score,ast_ngram_terminal_n_2_score,ast_ngram_terminal_n_3_score,ast_ngram_terminal_n_4_score,ast_ngram_terminal_n_5_score,ast_ngram_scoring_n_2_score,ast_ngram_scoring_n_3_score,ast_ngram_scoring_n_4_score,ast_ngram_scoring_n_5_score
0,0,interactive-beta.pddl,game-0,medium-objects-room-v1,game-0,1,1,1.0,1,1.0,...,0.969177,0.979573,0.965285,0.976289,0.977376,0.980148,0.915272,0.960175,0.999188,1.000000
1,1,interactive-beta.pddl,game-2,many-objects-room-v1,game-2,1,1,1.0,1,1.0,...,0.974875,0.978991,0.961082,0.972691,0.971810,0.980943,0.895362,0.910697,0.963311,0.973001
2,2,interactive-beta.pddl,game-3,many-objects-room-v1,game-3,1,1,1.0,1,1.0,...,0.954233,0.972808,0.000000,0.000000,0.000000,0.000000,0.841097,0.898279,0.946597,0.955042
3,3,interactive-beta.pddl,game-5,few-objects-room-v1,game-5,1,1,1.0,1,1.0,...,0.986178,0.982877,0.000000,0.000000,0.000000,0.000000,0.915272,0.960175,0.999188,1.000000
4,4,interactive-beta.pddl,game-6,medium-objects-room-v1,game-6,1,1,1.0,1,1.0,...,0.984002,0.979782,0.000000,0.000000,0.000000,0.000000,0.964905,0.971973,0.972563,0.962818


In [4]:
REAL_GAME_INDICES_TO_INCLUDE = [
    0, 4, 6, 7, 11,
    14, 17, 23, 26, 28,
    31, 32, 35, 37, 40,
    41, 42, 45, 49, 51,
    52, 55, 58, 59, 64,
    74, 88, 90, 94, 96,
]

model_key = 'max_exemplar_preferences_by_bcs_with_expected_values'
model_spec = latest_model_paths.MAP_ELITES_MODELS[model_key]
model = typing.cast(MAPElitesSampler, model_spec.load())

key_to_real_game_index = defaultdict(list)
real_game_index_to_key = {}
real_game_fitness_scores = []
ALL_REAL_GAME_KEYS = []
for i, ast in enumerate(game_asts):
    fitness_score, features = model._score_proposal(ast, return_features=True)  # type: ignore
    real_game_fitness_scores.append(fitness_score)
    key = model._features_to_key(ast, features)
    key_to_real_game_index[key].append(i)
    real_game_index_to_key[i] = key
    ALL_REAL_GAME_KEYS.append(key)




2024-05-29 16:19:05 - compile_predicate_statistics_full_database - INFO     - Loading data from files with query timeout 15
2024-05-29 16:19:26 - compile_predicate_statistics_full_database - INFO     - Creating DuckDB table...
2024-05-29 16:19:28 - compile_predicate_statistics_full_database - INFO     - Creating data table indices...
2024-05-29 16:19:49 - compile_predicate_statistics_full_database - INFO     - Loaded data, found 843825 rows


# Load back-translated descriptions

In [5]:
TRANSLATIONS_DIR = './backtranslations'
TRANSLATION_DATE = '2024_01_12'

with open(f'{TRANSLATIONS_DIR}/human_games_translations_split_{TRANSLATION_DATE}.json') as f:
    human_game_texts = json.load(f)

with open(f'{TRANSLATIONS_DIR}/human_cell_archive_games_translations_split_{TRANSLATION_DATE}.json') as f:
    human_cell_archive_game_texts = json.load(f)

with open(f'{TRANSLATIONS_DIR}/novel_archive_cell_games_translations_split_{TRANSLATION_DATE}.json') as f:
    novel_archive_cell_game_texts = json.load(f)

# Use one of the functions below to print games

In [6]:
def print_real_participant_game(index: typing.Optional[int] = None, with_backtranslation_only: bool = False):
    if index is None:
        if with_backtranslation_only:
            index = np.random.choice(REAL_GAME_INDICES_TO_INCLUDE)

        else:
            index = np.random.choice(len(human_game_texts))

    print('Game program:\n')
    print(ast_printer.ast_to_string(game_asts[index], '\n'))
    print('\n', '=' * 80, '\n')
    if index in REAL_GAME_INDICES_TO_INCLUDE:
        key = ALL_REAL_GAME_KEYS[index]
        translation = human_game_texts[str(key)]
        print('Game text (GPT-4 back-translated):\n')
        print(translation)

    else:
        print('Game was not back-translated')


def print_model_game(key: typing.Optional[KeyTypeAnnotation] = None, with_backtranslation_only: bool = False):
    if key is None:
        if with_backtranslation_only:
            all_translated_keys = list(human_cell_archive_game_texts.keys()) + list(novel_archive_cell_game_texts.keys())
            key_str = np.random.choice(all_translated_keys)
            key = literal_eval(key_str)

        else:
            key_list = list(model.population.keys())
            key_index = np.random.choice(len(key_list))
            key = key_list[key_index]
    
    print('Game program:\n')
    print(ast_printer.ast_to_string(model.population[key], '\n'))
    print('\n', '=' * 80, '\n')
    key_str = str(key)
    if key_str in human_cell_archive_game_texts:
        translation = human_cell_archive_game_texts[key_str]
        print('Game text (GPT-4 back-translated):\n')
        print(translation)

    elif key_str in novel_archive_cell_game_texts:
        translation = novel_archive_cell_game_texts[key_str]
        print('Game text (GPT-4 back-translated):\n')
        print(translation)

    else:
        print('Game was not back-translated')





In [7]:
print_real_participant_game(with_backtranslation_only=True)

Game program:

(define (game game-106) (:domain few-objects-room-v1)
(:constraints
  (and
    (preference throwInBin
      (exists (?b - ball ?h - hexagonal_bin)
        (then
          (once (agent_holds ?b))
          (hold (and (not (agent_holds ?b)) (in_motion ?b)))
          (once (and (not (in_motion ?b)) (in ?h ?b)))
       )
     )
   )
    (preference throwAttempt
      (exists (?b - ball)
        (then
          (once (agent_holds ?b))
          (hold (and (not (agent_holds ?b)) (in_motion ?b)))
          (once (not (in_motion ?b)))
       )
     )
   )
 )
)
(:terminal
  (or
    (>= (total-score) 6)
    (>= (count throwAttempt) 15)
 )
)
(:scoring
  (count throwInBin)
)
)


Game text (GPT-4 back-translated):

Gameplay: hold balls and throw them so that they stop moving, aiming to get them inside a hexagonal bin.

Terminal: the game ends once you've scored 6 or more points, or after you've made at least 15 throw attempts.

Scoring: you get 1 point for each ball that stops movin

In [8]:
print_model_game(with_backtranslation_only=True)

Game program:

(define (game evo-8087-311-0) (:domain few-objects-room-v1)
(:setup
  (forall (?v0 - dodgeball)
    (game-optional
      (near desk ?v0)
   )
 )
)
(:constraints
  (and
    (preference preference0
      (exists (?v1 - dodgeball ?v0 - teddy_bear)
        (then
          (once (agent_holds ?v1))
          (hold (and (in_motion ?v1) (not (agent_holds ?v1)) (not (agent_holds ?v0)) (touch south_wall ?v1)))
          (once (not (in_motion ?v1)))
       )
     )
   )
 )
)
(:terminal
  (>= (total-time) 60)
)
(:scoring
  (count preference0)
)
)


Game text (GPT-4 back-translated):

Setup: Place a dodgeball near the desk before starting the game.

Gameplay: Throw the dodgeball so that it touches the south wall while it's moving, and make sure you're not holding the dodgeball or a teddy bear when it's in motion.

Terminal: The game ends after 60 seconds.

Scoring: You get 1 point for each time you successfully throw the dodgeball and it touches the south wall and then stops moving.
